In [125]:
import cv2
import torch

In [126]:
# 연산
import math

In [127]:
import matplotlib.pyplot as plt
import matplotlib
plt.plot(range(1,10), range(1,10))
plt.show()

In [128]:
# 학습모델 호출
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('ultralytics/yolov5', 'custom', path="../train/best.pt", force_reload=True)
model.to(device)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\haiiron/.cache\torch\hub\master.zip
YOLOv5  2024-5-6 Python-3.11.7 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 48, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
  

In [129]:
# FOV
horizontal_FOV = 77
vertical_FOV = 69
# coordinate
lat = 36.168268
lng = 128.468008

# altitude (meter)
altitude = 100

In [133]:
def image_to_gps(x_image, y_image, image_width, image_height):
    # 이미지 상 중간 좌표
    center_x = image_width / 2
    center_y = image_height / 2
    
    # 이미지에서 객체까지의 거리 계산 (단위: 픽셀)
    distance_x = x_image - center_x
    distance_y = y_image - center_y
    
    # 이미지에서 객체까지의 거리에 대한 FOV를 사용하여 실제 거리 계산 (단위: 미터)
    horizontal_real = altitude / math.tan(math.radians(horizontal_FOV / 2)) * math.tan(math.radians(distance_x * (horizontal_FOV / 2) / center_x))
    vertical_real = altitude / math.tan(math.radians(vertical_FOV / 2)) * math.tan(math.radians(distance_x * (vertical_FOV / 2) / center_x))
    # 이미지에서 객체까지의 각도 계산
    angle_x = math.degrees(math.atan(distance_x / center_x))
    angle_y = math.degrees(math.atan(distance_y / center_y))
    
    # 객체의 위도 및 경도 계산
    object_latitude = lat + math.degrees(math.asin(horizontal_FOV * math.cos(math.radians(angle_y)) / 6371.0))
    object_longitude = lng + math.degrees(math.asin(vertical_FOV * math.sin(math.radians(angle_x)) / (6371.0 * math.cos(math.radians(lat)))))
    
    # 소수점 이하 여섯 자리까지만 반환
    object_latitude = round(object_latitude, 6)
    object_longitude = round(object_longitude, 6)
    
    return object_latitude, object_longitude

In [134]:
case_middle = cv2.imread("resoruces/DJI_01.JPG",cv2.IMREAD_COLOR)
h, w, _ = case_middle.shape

image_center_x = (w / 2)
image_center_y = (h / 2)

results = model(case_middle)

for result in results.xyxy[0]:
    xmin, ymin, xmax, ymax, conf, cls = result
if conf >= 0.65:
        cv2.rectangle(case_middle, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 15, 0), 3)
        cv2.putText(case_middle, "tank", (int(xmin), int(ymin) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        # 좌표
        x = (xmin + xmax) / 2
        y = (ymin + ymax) / 2

print(f"IMAGE width = {w}, height = {h}")
print(f"TARGT X_cord = {image_center_x - x}, Y_cord = {image_center_y - y}")

IMAGE width = 4000, height = 2250
TARGT X_cord = 21.6583251953125, Y_cord = -342.758544921875


In [135]:
plt.subplot(1, 1, 1)
plt.title("normal target")
plt.imshow(case_middle)
plt.xticks([]), plt.yticks([])
print("Tank coordinates")
print(image_to_gps(x,y,w,h))
plt.show()

Tank coordinates
(36.581206, 128.462218)
